In [ ]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [ ]:
import torch
import os
from utils.weight_sharing import *
from data.imagenette import *
from data.utils.download import *
from data.utils.imagenet_utils import *

BATCH_SIZE = 32

In [ ]:
net = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

dat = ImagenetteDataset(BATCH_SIZE, os.path.join(PATH_PREFIX, 'data/imagenette/'), val_split=0.3)


In [ ]:
print('test ', len(dat.test_dataset))
print('val', len(dat.valid_dataset))
print('train', len(dat.train_dataset))

In [ ]:
acc = get_accuracy(net, dat.test_dl, 'cpu', topk=1)
#acc - 0.01 * acc
acc

In [ ]:
acc

In [ ]:
net

In [ ]:
ws_controller = WeightShare(net, lambda: get_accuracy(net, dat.test_dl, 'cpu', topk=1))
ws_controller.print_layers_info()

In [ ]:
ws_controller.model_layers[0].weight.dtype

In [ ]:
len(ws_controller.model_layers)

In [ ]:
ws_controller.share([70, 96, 103, 120, 120, 120, 31, 120, 96, 120, 120, 117, 12, 55, 120, 63, 23, 73, 120, 28, 74, 12, 55, 106, 120, 102, 120, 13, 68, 65, 48, 69, 120, 22, 120, 71, 21, 120, 120, 57, 53, 91, 120, 120, 18, 120, 93, 32, 63, 120, 120, 120, 114], verbose=True, prec_reduct=['f4' for _ in ws_controller.model_layers], clust_alg='minibatch-kmeans')

In [ ]:
classes = json.load(open(os.path.join(PATH_PREFIX, '/data/imagenette/imagenet_class_index.json')))
classes_arr = []

for i in range(len(classes.keys())):
    classes_arr.append(classes[f'{i}'])

output = net(dat.test_dl.__getitem__(1)[0].unsqueeze(0))

probabilities = torch.nn.functional.softmax(output[0], dim=0)

top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(classes_arr[top5_catid[i]], top5_prob[i].item(), top5_catid[i])